In [24]:
import pandas as pd
import numpy as np
import geopandas as gpd
import branca.colormap as cm
import folium

In [35]:
filepath = '../../data/processed/Brazil/election_data/2018/president/turn_2/matched_census/by_state/RS/IS_0.99946/data/weightening_area_aggr.csv'
meshpath = '../../data/interim/Brazil/census_data/2010/weightening_area/digital_mesh/RS.geojson'

In [51]:
data = pd.read_csv(filepath)
digital_mesh = gpd.read_file(meshpath)

In [52]:
data = digital_mesh.merge(data, on='Cod_ap', how='right')
print('Weightening areas: {}'.format(len(data)))
data.head()

Weightening areas: 825


,id,CD_GEOCODM,Cod_ap,NM_MUNICIP,geometry,SG_ UF,NM_MUNICIPIO,CD_MUNICIPIO,COD_LOCALIDADE_IBGE,NR_ZONA,...,lev_dist,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_ELEITORES_BIOMETRIA_NH,Branco,Nulo,JAIR BOLSONARO,FERNANDO HADDAD,precision
0,0,4300034,4.300034e+12,ACEGUÁ,"POLYGON ((-53.97581 -31.77023, -53.96848 -31.7...",RS,ACEGUÁ,88986,4300034.0,"[7, 7, 7]",...,1.0,3653,2974,679,655.0,109,101,1739,1025,"['TSE', 'TSE', 'TSE']"
1,1,4300059,4.300059e+12,ÁGUA SANTA,"POLYGON ((-52.02139 -28.30491, -52.02744 -28.3...",RS,SANTA CECÍLIA DO SUL,89247,4316733.0,"[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",...,1.0,3062,2682,380,439.0,68,129,1717,768,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS..."
2,2,4300109,4.300109e+12,AGUDO,"POLYGON ((-53.21638 -29.72531, -53.21796 -29.7...",RS,AGUDO,85014,4300109.0,"[119, 119, 119, 119, 119, 119, 119, 119, 119, ...",...,1.0,11908,10898,1010,814.0,360,546,6043,3949,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS..."
3,3,4300208,4.300208e+12,AJURICABA,"POLYGON ((-53.62610 -28.21418, -53.62811 -28.2...",RS,AJURICABA,85030,4300208.0,"[23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23]",...,1.0,5883,5009,874,573.0,87,171,3116,1635,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS..."
4,4,4300307,4.300307e+12,ALECRIM,"POLYGON ((-54.76596 -27.73506, -54.77637 -27.7...",RS,ALECRIM,85057,4300307.0,"[102, 102, 102, 102, 102, 102, 102, 102, 102, ...",...,1.0,5181,4587,594,1113.0,94,229,2355,1909,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS..."


In [53]:
data['BOLSONARO_VOTE_SHARE'] = 100*data['JAIR BOLSONARO']/data['QT_COMPARECIMENTO']
data['HADDAD_VOTE_SHARE'] = 100*data['FERNANDO HADDAD']/data['QT_COMPARECIMENTO']
who_won = [-bolsonaro if bolsonaro > haddad else haddad for bolsonaro, haddad in zip(data['BOLSONARO_VOTE_SHARE'],data['HADDAD_VOTE_SHARE'])]
data['WHO_WON'] = who_won
data.head()

,id,CD_GEOCODM,Cod_ap,NM_MUNICIP,geometry,SG_ UF,NM_MUNICIPIO,CD_MUNICIPIO,COD_LOCALIDADE_IBGE,NR_ZONA,...,QT_ABSTENCOES,QT_ELEITORES_BIOMETRIA_NH,Branco,Nulo,JAIR BOLSONARO,FERNANDO HADDAD,precision,BOLSONARO_VOTE_SHARE,HADDAD_VOTE_SHARE,WHO_WON
0,0,4300034,4.300034e+12,ACEGUÁ,"POLYGON ((-53.97581 -31.77023, -53.96848 -31.7...",RS,ACEGUÁ,88986,4300034.0,"[7, 7, 7]",...,679,655.0,109,101,1739,1025,"['TSE', 'TSE', 'TSE']",58.473436,34.465367,-58.473436
1,1,4300059,4.300059e+12,ÁGUA SANTA,"POLYGON ((-52.02139 -28.30491, -52.02744 -28.3...",RS,SANTA CECÍLIA DO SUL,89247,4316733.0,"[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",...,380,439.0,68,129,1717,768,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS...",64.019389,28.635347,-64.019389
2,2,4300109,4.300109e+12,AGUDO,"POLYGON ((-53.21638 -29.72531, -53.21796 -29.7...",RS,AGUDO,85014,4300109.0,"[119, 119, 119, 119, 119, 119, 119, 119, 119, ...",...,1010,814.0,360,546,6043,3949,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS...",55.450541,36.236007,-55.450541
3,3,4300208,4.300208e+12,AJURICABA,"POLYGON ((-53.62610 -28.21418, -53.62811 -28.2...",RS,AJURICABA,85030,4300208.0,"[23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23]",...,874,573.0,87,171,3116,1635,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS...",62.208026,32.641246,-62.208026
4,4,4300307,4.300307e+12,ALECRIM,"POLYGON ((-54.76596 -27.73506, -54.77637 -27.7...",RS,ALECRIM,85057,4300307.0,"[102, 102, 102, 102, 102, 102, 102, 102, 102, ...",...,594,1113.0,94,229,2355,1909,"['TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TSE', 'TS...",51.340746,41.617615,-51.340746


In [55]:
linear = cm.LinearColormap( ['#2166ac','#59a2cb','#bedbea','#F9F9F3','#facab2', '#df745b', '#b2182c'],
                          vmin=-100, vmax=100)
m = folium.Map(location=[-14.2350, -51.9253], zoom_start=5, tiles='cartodbpositron')
folium.GeoJson(data=data,
               name='RS Votes',
               tooltip=folium.GeoJsonTooltip(fields=['NM_MUNICIP', 'BOLSONARO_VOTE_SHARE', 'HADDAD_VOTE_SHARE'],
                                             aliases=['<div style="background-color: lightyellow; color: black; padding: 3px; border: 2px solid black; border-radius: 3px;">'+item+'</div>' for item in ['City','BOLSONARO_VOTE_SHARE', 'HADDAD_VOTE_SHARE']],
                                             labels=True,
                                             sticky=True),
                  style_function=lambda city: {'fillColor': linear(city['properties']['WHO_WON']),
                                           'color': 'black',
                                           'fillOpacity': 0.7,
                                           'lineOpacity':0.1,
                                           'weight': 0.3
                                          }
              ).add_to(m)
m.add_child(linear)
m.save('../../reports/figures/votos_turno_2_RS.html')